# Position Categorization (User-Friendly Notebook)

This notebook categorizes genomic features by their position (e.g., intergenic, intragenic, antisense).

## 1) Configure inputs below
Edit the variables in the next cell (paths, column names, and optional parameters), then run all cells top-to-bottom.


In [ ]:
# =============================
# Configuration (edit here)
# =============================

# Input files
SRNA_FILE = "path/to/srna_table.csv"          # e.g., srna coordinates (id, start, end, strand)
GENE_FILE = "path/to/gene_annotation.csv"    # e.g., gene coordinates (gene_id, start, end, strand)

# Output folder (created if missing)
OUTPUT_DIR = "outputs"

# Column names (change only if your files use different headers)
COL_ID = "exon_id"       # sRNA identifier column
COL_START = "start"
COL_END = "end"
COL_STRAND = "strand"
COL_GENE_ID = "gene_id"  # gene identifier column in gene table

# Optional: control what happens when overlaps exist
PREFER_SMALLEST_OVERLAP = True  # if multiple genes overlap, pick the one with smallest overlap range

import os
os.makedirs(OUTPUT_DIR, exist_ok=True)


In [ ]:
import pandas as pd
import numpy as np

def load_table(path: str) -> pd.DataFrame:
    """Load a CSV/TSV table based on file extension."""
    if path.lower().endswith('.tsv') or path.lower().endswith('.txt'):
        return pd.read_csv(path, sep='\t')
    return pd.read_csv(path)

def ensure_required_columns(df: pd.DataFrame, required: list[str], table_name: str) -> None:
    missing = [c for c in required if c not in df.columns]
    if missing:
        raise ValueError(f"{table_name} is missing required columns: {missing}. Found: {list(df.columns)}")

def add_range_size(df: pd.DataFrame, start_col: str, end_col: str, out_col: str = 'range_size') -> pd.DataFrame:
    """Return a copy with an extra range size column, avoiding SettingWithCopyWarning."""
    out = df.copy()
    out[out_col] = out[end_col] - out[start_col]
    return out


In [2]:
def read_gtf(gtf_path):
    gtf_columns = ["seqname", "source", "feature", "start", "end", "score", "strand", "frame", "attribute"]
    gtf_df = pd.read_csv(gtf_path, sep='\t', comment='#', header=None, names=gtf_columns)
    return gtf_df


In [3]:
match = read_gtf('BMB.gtf')

match = match[['feature', 'attribute', "start", "end", "score", "strand"]]

# Step 1: Filter rows that contain "transcript" in the feature column
transcript_data = match[match['feature'] == 'exon']
#transcript_data = match[match['feature'] == "transcript"]


# Step 2: Extract the desired key-value pairs
# Define the keys we're interested in
inner_keys = ['exon_id']
#inner_keys = ["locus_tag"]
outer_keys = ['start', 'end', 'strand']

# Function to extract values based on keys
def extract_values(row, inner_keys, outer_keys=None):

    if outer_keys:
        values = {key: None for key in inner_keys+outer_keys}
        for key in outer_keys:
            values[key] = row[key]
    else:
        values = {key: None for key in inner_keys}

    attribute = row['attribute']
    parts = attribute.split('; ')
    for part in parts:
        for key in inner_keys:
            if part.startswith(key + ' '):
                values[key] = part.split('"')[1]

    return values


# Apply the extraction function to each attribute in the filtered data
extracted_data = transcript_data.apply(lambda row: extract_values(row, inner_keys, outer_keys), axis=1)

# Convert the list of dictionaries into a DataFrame
result = pd.DataFrame(list(extracted_data.values))


In [4]:
# Rename the 'score' column to 'length'
result.rename(columns={'score': 'length'}, inplace=True)

# Create a new column 'start_end' by combining 'start' and 'end'
result['length'] =  result['end'] - result['start']


In [5]:
#result = result[~result['exon_id'].str.startswith(('id'))]


In [7]:
filtered_results = results[results.iloc[:, 0].fillna('').astype(str).str.startswith(('srn_', 'SABB')) == True]

# Display the filtered DataFrame
filtered_results

,exon_id,start,end,strand,length
0,srn_0010_sRNA412,312,511,+,199
1,SABB_RS00005,516,1877,+,1361
2,SABB_RS15365,1453,2376,-,923
3,SABB_RS00010,2155,3288,+,1133
4,SABB_RS15360,2846,3565,+,719
...,...,...,...,...,...
3750,SABB_RS15335,2975537,2976256,-,719
3751,SABB_RS15340,2976256,2978133,-,1877
3752,SABB_RS15345,2978200,2979579,-,1379
3753,SABB_RS15350,2979723,2980076,-,353


In [8]:
# Ensure the first column is treated as a string and handle NaN values
filtered_results.iloc[:, 0] = filtered_results.iloc[:, 0].fillna('').astype(str)

# Filter rows starting with 'srn_'
srn_results = filtered_results[filtered_results.iloc[:, 0].str.startswith('srn_')]

# Filter rows starting with ''
gene_results = filtered_results[filtered_results.iloc[:, 0].str.startswith('SABB')]

# Display the results
print("Rows starting with 'srn_':")
print(srn_results)

print("\nRows starting with 'SABB':")
print(gene_results)


Rows starting with 'srn_':
                   exon_id    start      end strand  length
0         srn_0010_sRNA412      312      511      +     199
9           srn_0020_sRNA1     8413     8585      -     172
14        srn_0040_Sau6817    14262    14353      -      91
17           srn_0050_Teg1    15795    16036      +     241
24          srn_0060_Sau65    21713    21771      -      58
...                    ...      ...      ...    ...     ...
3744  srn_2020_sRNA171sRNA  2974257  2974532      +     275
3745      srn_2020_sRNA171  2974289  2974352      -      63
3746      srn_1930.2_rsaOF  2974383  2974438      -      55
3747      srn_4610_sRNA377  2974437  2974532      -      95
3749      srn_5130_Sau6728  2975498  2975576      +      78

[685 rows x 5 columns]

Rows starting with 'SABB':
           exon_id    start      end strand  length
1     SABB_RS00005      516     1877      +    1361
2     SABB_RS15365     1453     2376      -     923
3     SABB_RS00010     2155     3288      +  

In [9]:
srn_results['location'] = None
#srn_results['Interception'] = None
srn_results


<ipython-input-9-d3521e58c89e>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  srn_results['location'] = None


,exon_id,start,end,strand,length,location
0,srn_0010_sRNA412,312,511,+,199,None
9,srn_0020_sRNA1,8413,8585,-,172,None
14,srn_0040_Sau6817,14262,14353,-,91,None
17,srn_0050_Teg1,15795,16036,+,241,None
24,srn_0060_Sau65,21713,21771,-,58,None
...,...,...,...,...,...,...
3744,srn_2020_sRNA171sRNA,2974257,2974532,+,275,None
3745,srn_2020_sRNA171,2974289,2974352,-,63,None
3746,srn_1930.2_rsaOF,2974383,2974438,-,55,None
3747,srn_4610_sRNA377,2974437,2974532,-,95,None


In [10]:
########################
#
# Only consider cases that are fully contained
#
#
######################
'''
# Ensure numeric columns are properly formatted
srn_results[['start', 'end']] = srn_results[['start', 'end']].astype(int)
gene_results[['start', 'end']] = gene_results[['start', 'end']].astype(int)

# Initialize the new columns
srn_results['location'] = None
srn_results['gene_id'] = None

# Loop through each row in gene_results and compare range and strand
for _, gene_row in gene_results.iterrows():
    mask = (
        (srn_results['start'] >= gene_row['start']) &
        (srn_results['end'] <= gene_row['end'])
    )
    # Check strand similarity and update 'location' and 'gene_id'
    srn_results.loc[mask & (srn_results['strand'] == gene_row['strand']), 'location'] = 'Intragenic'
    srn_results.loc[mask & (srn_results['strand'] != gene_row['strand']), 'location'] = 'Antisense'
    srn_results.loc[mask, 'gene_id'] = gene_row.iloc[0]  # Add gene name from the first column

# Display the updated DataFrame
print(srn_results)

# Count the occurrences of each unique value in 'location'
location_counts = srn_results['location'].value_counts(dropna=True).reset_index()

# Rename columns for clarity
location_counts.columns = ['location', 'count']

# Display the counts
print(location_counts)
'''

"\n# Ensure numeric columns are properly formatted\nsrn_results[['start', 'end']] = srn_results[['start', 'end']].astype(int)\ngene_results[['start', 'end']] = gene_results[['start', 'end']].astype(int)\n\n# Initialize the new columns\nsrn_results['location'] = None\nsrn_results['gene_id'] = None\n\n# Loop through each row in gene_results and compare range and strand\nfor _, gene_row in gene_results.iterrows():\n    mask = (\n        (srn_results['start'] >= gene_row['start']) &\n        (srn_results['end'] <= gene_row['end'])\n    )\n    # Check strand similarity and update 'location' and 'gene_id'\n    srn_results.loc[mask & (srn_results['strand'] == gene_row['strand']), 'location'] = 'Intragenic'\n    srn_results.loc[mask & (srn_results['strand'] != gene_row['strand']), 'location'] = 'Antisense'\n    srn_results.loc[mask, 'gene_id'] = gene_row.iloc[0]  # Add gene name from the first column\n\n# Display the updated DataFrame\nprint(srn_results)\n\n# Count the occurrences of each uniq

In [11]:
########################
#
# Accept 90% overlap for Antisense and Intragenic
#
#
######################

# Ensure numeric columns are properly formatted
srn_results[['start', 'end']] = srn_results[['start', 'end']].astype(int)
gene_results[['start', 'end']] = gene_results[['start', 'end']].astype(int)

# Initialize the new columns
srn_results['location'] = None
srn_results['gene_id'] = None

# Loop through each row in gene_results and apply logic
for _, gene_row in gene_results.iterrows():
    # Calculate mask for range inclusion
    mask = (
        (srn_results['start'] >= gene_row['start']) &
        (srn_results['end'] <= gene_row['end'])
    )

    # Calculate mask for 90% overlap
    srn_length = srn_results['end'] - srn_results['start']
    overlap_start = srn_results['start'].clip(lower=gene_row['start'])
    overlap_end = srn_results['end'].clip(upper=gene_row['end'])
    overlap_length = (overlap_end - overlap_start).clip(lower=0)
    overlap_ratio = overlap_length / srn_length

    overlap_mask = overlap_ratio >= 0.9

    # Apply logic for Intragenic and Antisense
    intragenic_mask = (mask | overlap_mask) & (srn_results['strand'] == gene_row['strand'])
    antisense_mask = (mask | overlap_mask) & (srn_results['strand'] != gene_row['strand'])

    srn_results.loc[intragenic_mask, 'location'] = 'Intragenic'
    srn_results.loc[antisense_mask, 'location'] = 'Antisense'
    srn_results.loc[mask | overlap_mask, 'gene_id'] = gene_row.iloc[0]

# Display the updated DataFrame
print(srn_results)

<ipython-input-11-a47227bdb64d>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  srn_results[['start', 'end']] = srn_results[['start', 'end']].astype(int)
<ipython-input-11-a47227bdb64d>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gene_results[['start', 'end']] = gene_results[['start', 'end']].astype(int)
<ipython-input-11-a47227bdb64d>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveat

                   exon_id    start      end strand  length   location  \
0         srn_0010_sRNA412      312      511      +     199       None   
9           srn_0020_sRNA1     8413     8585      -     172  Antisense   
14        srn_0040_Sau6817    14262    14353      -      91       None   
17           srn_0050_Teg1    15795    16036      +     241       None   
24          srn_0060_Sau65    21713    21771      -      58  Antisense   
...                    ...      ...      ...    ...     ...        ...   
3744  srn_2020_sRNA171sRNA  2974257  2974532      +     275       None   
3745      srn_2020_sRNA171  2974289  2974352      -      63       None   
3746      srn_1930.2_rsaOF  2974383  2974438      -      55       None   
3747      srn_4610_sRNA377  2974437  2974532      -      95       None   
3749      srn_5130_Sau6728  2975498  2975576      +      78       None   

           gene_id  
0             None  
9     SABB_RS00030  
14            None  
17            None  
24    

In [13]:
########################
#
# Accept 90% overlap for 5' UTR and 3' UTR
#
#
######################

# Ensure numeric columns are properly formatted
srn_results[['start', 'end']] = srn_results[['start', 'end']].astype(int)
gene_results[['start', 'end']] = gene_results[['start', 'end']].astype(int)

# Filter rows in srn_results that don't already have a location assigned
unclassified_srn = srn_results[srn_results['location'].isna()]

# Loop through each row in gene_results to check for Upstream and Downstream
for _, gene_row in gene_results.iterrows():
    # Filter by same strand first
    same_strand_mask = unclassified_srn['strand'] == gene_row['strand']
    same_strand_srn = unclassified_srn[same_strand_mask]

    # Calculate lengths for overlap calculation
    srn_length = same_strand_srn['end'] - same_strand_srn['start']

    # Upstream range (90% overlap logic)
    upstream_overlap_start = same_strand_srn['start'].clip(lower=gene_row['start'] - 150)
    upstream_overlap_end = same_strand_srn['end'].clip(upper=gene_row['start'])
    upstream_overlap_length = (upstream_overlap_end - upstream_overlap_start).clip(lower=0)
    upstream_overlap_ratio = upstream_overlap_length / srn_length

    upstream_mask = upstream_overlap_ratio >= 0.9

    # Downstream range (90% overlap logic)
    downstream_overlap_start = same_strand_srn['start'].clip(lower=gene_row['end'])
    downstream_overlap_end = same_strand_srn['end'].clip(upper=gene_row['end'] + 150)
    downstream_overlap_length = (downstream_overlap_end - downstream_overlap_start).clip(lower=0)
    downstream_overlap_ratio = downstream_overlap_length / srn_length

    downstream_mask = downstream_overlap_ratio >= 0.9

    # Apply Upstream and classify as 5' UTR or 3' UTR based on strand direction
    upstream_5utr_mask = upstream_mask & (same_strand_srn['strand'] == '+') & (gene_row['strand'] == '+')
    upstream_3utr_mask = upstream_mask & (same_strand_srn['strand'] == '-') & (gene_row['strand'] == '-')

    srn_results.loc[same_strand_srn.index[upstream_5utr_mask], 'location'] = "5' UTR"
    srn_results.loc[same_strand_srn.index[upstream_3utr_mask], 'location'] = "3' UTR"
    srn_results.loc[same_strand_srn.index[upstream_mask], 'gene_id'] = gene_row.iloc[0]

    # Apply Downstream and classify as 5' UTR or 3' UTR based on strand direction
    downstream_3utr_mask = downstream_mask & (same_strand_srn['strand'] == '+') & (gene_row['strand'] == '+')
    downstream_5utr_mask = downstream_mask & (same_strand_srn['strand'] == '-') & (gene_row['strand'] == '-')

    srn_results.loc[same_strand_srn.index[downstream_3utr_mask], 'location'] = "3' UTR"
    srn_results.loc[same_strand_srn.index[downstream_5utr_mask], 'location'] = "5' UTR"
    srn_results.loc[same_strand_srn.index[downstream_mask], 'gene_id'] = gene_row.iloc[0]

# Display the updated DataFrame
srn_results


<ipython-input-13-1d047a1c2a66>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  srn_results[['start', 'end']] = srn_results[['start', 'end']].astype(int)
<ipython-input-13-1d047a1c2a66>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gene_results[['start', 'end']] = gene_results[['start', 'end']].astype(int)


,exon_id,start,end,strand,length,location,gene_id
0,srn_0010_sRNA412,312,511,+,199,None,None
9,srn_0020_sRNA1,8413,8585,-,172,Antisense,SABB_RS00030
14,srn_0040_Sau6817,14262,14353,-,91,None,None
17,srn_0050_Teg1,15795,16036,+,241,None,None
24,srn_0060_Sau65,21713,21771,-,58,Antisense,SABB_RS00080
...,...,...,...,...,...,...,...
3744,srn_2020_sRNA171sRNA,2974257,2974532,+,275,None,None
3745,srn_2020_sRNA171,2974289,2974352,-,63,None,None
3746,srn_1930.2_rsaOF,2974383,2974438,-,55,None,None
3747,srn_4610_sRNA377,2974437,2974532,-,95,None,None


In [17]:
# Ensure numeric columns are properly formatted
srn_results.loc[:, ['start', 'end']] = srn_results[['start', 'end']].astype(int)
gene_results.loc[:, ['start', 'end']] = gene_results[['start', 'end']].astype(int)

# Filter rows in srn_results that don't already have a location assigned
unclassified_srn = srn_results[srn_results['location'].isna()].copy()

# Loop through each row in gene_results to check for Upstream and Downstream
for _, gene_row in gene_results.iterrows():
    # Filter by same strand
    same_strand_mask = unclassified_srn['strand'] == gene_row['strand']
    same_strand_srn = unclassified_srn[same_strand_mask].copy()

    # Calculate lengths for overlap calculation
    srn_length = same_strand_srn['end'] - same_strand_srn['start']

    # Gene Overlap (for intragenic check)
    gene_overlap_start = same_strand_srn['start'].clip(lower=gene_row['start'])
    gene_overlap_end = same_strand_srn['end'].clip(upper=gene_row['end'])
    gene_overlap_length = (gene_overlap_end - gene_overlap_start).clip(lower=0)
    gene_overlap_ratio = gene_overlap_length / srn_length

    # Upstream range
    upstream_overlap_start = same_strand_srn['start'].clip(lower=gene_row['start'] - 150)
    upstream_overlap_end = same_strand_srn['end'].clip(upper=gene_row['start'])
    upstream_overlap_length = (upstream_overlap_end - upstream_overlap_start).clip(lower=0)
    upstream_overlap_ratio = upstream_overlap_length / srn_length

    # Downstream range
    downstream_overlap_start = same_strand_srn['start'].clip(lower=gene_row['end'])
    downstream_overlap_end = same_strand_srn['end'].clip(upper=gene_row['end'] + 150)
    downstream_overlap_length = (downstream_overlap_end - downstream_overlap_start).clip(lower=0)
    downstream_overlap_ratio = downstream_overlap_length / srn_length

    ## Logic for Classification

    ### Upstream Classification
    # 5' UTR/intragenic (++)
    upstream_5utr_intragenic_mask = (upstream_overlap_ratio >= 0.5) & (gene_overlap_ratio >= 0.1) & (same_strand_srn['strand'] == '+') & (gene_row['strand'] == '+')
    # 3' UTR/intragenic (--)
    upstream_3utr_intragenic_mask = (upstream_overlap_ratio >= 0.5) & (gene_overlap_ratio >= 0.1) & (same_strand_srn['strand'] == '-') & (gene_row['strand'] == '-')

    srn_results.loc[same_strand_srn.index[upstream_5utr_intragenic_mask], 'location'] = "5' UTR"
    srn_results.loc[same_strand_srn.index[upstream_3utr_intragenic_mask], 'location'] = "3' UTR"
    srn_results.loc[same_strand_srn.index[upstream_5utr_intragenic_mask | upstream_3utr_intragenic_mask], 'gene_id'] = gene_row.iloc[0]

    ### Downstream Classification
    # 3' UTR/intragenic (++)
    downstream_3utr_intragenic_mask = (downstream_overlap_ratio >= 0.5) & (gene_overlap_ratio >= 0.1) & (same_strand_srn['strand'] == '+') & (gene_row['strand'] == '+')
    # 5' UTR/intragenic (--)
    downstream_5utr_intragenic_mask = (downstream_overlap_ratio >= 0.5) & (gene_overlap_ratio >= 0.1) & (same_strand_srn['strand'] == '-') & (gene_row['strand'] == '-')

    srn_results.loc[same_strand_srn.index[downstream_3utr_intragenic_mask], 'location'] = "3' UTR"
    srn_results.loc[same_strand_srn.index[downstream_5utr_intragenic_mask], 'location'] = "5' UTR"
    srn_results.loc[same_strand_srn.index[downstream_3utr_intragenic_mask | downstream_5utr_intragenic_mask], 'gene_id'] = gene_row.iloc[0]


In [18]:
# Ensure numeric columns are properly formatted
srn_results.loc[:, ['start', 'end']] = srn_results[['start', 'end']].astype(int)
gene_results.loc[:, ['start', 'end']] = gene_results[['start', 'end']].astype(int)

# Filter rows in srn_results that don't already have a location assigned
unclassified_srn = srn_results[srn_results['location'].isna()].copy()

# Loop through each row in gene_results to check for Upstream and Downstream
for _, gene_row in gene_results.iterrows():
    # Filter by same strand
    same_strand_mask = unclassified_srn['strand'] == gene_row['strand']
    same_strand_srn = unclassified_srn[same_strand_mask].copy()

    # Calculate lengths for overlap calculation
    srn_length = same_strand_srn['end'] - same_strand_srn['start']

    # Gene Overlap (for intragenic check)
    gene_overlap_start = same_strand_srn['start'].clip(lower=gene_row['start'])
    gene_overlap_end = same_strand_srn['end'].clip(upper=gene_row['end'])
    gene_overlap_length = (gene_overlap_end - gene_overlap_start).clip(lower=0)
    gene_overlap_ratio = gene_overlap_length / srn_length

    # Upstream range
    upstream_overlap_start = same_strand_srn['start'].clip(lower=gene_row['start'] - 150)
    upstream_overlap_end = same_strand_srn['end'].clip(upper=gene_row['start'])
    upstream_overlap_length = (upstream_overlap_end - upstream_overlap_start).clip(lower=0)
    upstream_overlap_ratio = upstream_overlap_length / srn_length

    # Downstream range
    downstream_overlap_start = same_strand_srn['start'].clip(lower=gene_row['end'])
    downstream_overlap_end = same_strand_srn['end'].clip(upper=gene_row['end'] + 150)
    downstream_overlap_length = (downstream_overlap_end - downstream_overlap_start).clip(lower=0)
    downstream_overlap_ratio = downstream_overlap_length / srn_length

    ## Logic for Classification

    ### Upstream Classification
    # 5' UTR/intragenic (++)
    upstream_5utr_intragenic_mask = (upstream_overlap_ratio >= 0.1) & (gene_overlap_ratio >= 0.5) & (same_strand_srn['strand'] == '+') & (gene_row['strand'] == '+')
    # 3' UTR/intragenic (--)
    upstream_3utr_intragenic_mask = (upstream_overlap_ratio >= 0.1) & (gene_overlap_ratio >= 0.5) & (same_strand_srn['strand'] == '-') & (gene_row['strand'] == '-')

    srn_results.loc[same_strand_srn.index[upstream_5utr_intragenic_mask], 'location'] = "Intragenic"
    srn_results.loc[same_strand_srn.index[upstream_3utr_intragenic_mask], 'location'] = "Intragenic"
    srn_results.loc[same_strand_srn.index[upstream_5utr_intragenic_mask | upstream_3utr_intragenic_mask], 'gene_id'] = gene_row.iloc[0]



    ### Downstream Classification
    # 3' UTR/intragenic (++)
    downstream_3utr_intragenic_mask = (downstream_overlap_ratio >= 0.1) & (gene_overlap_ratio >= 0.5) & (same_strand_srn['strand'] == '+') & (gene_row['strand'] == '+')
    # 5' UTR/intragenic (--)
    downstream_5utr_intragenic_mask = (downstream_overlap_ratio >= 0.1) & (gene_overlap_ratio >= 0.5) & (same_strand_srn['strand'] == '-') & (gene_row['strand'] == '-')

    srn_results.loc[same_strand_srn.index[downstream_3utr_intragenic_mask], 'location'] = "Intragenic"
    srn_results.loc[same_strand_srn.index[downstream_5utr_intragenic_mask], 'location'] = "Intragenic"
    srn_results.loc[same_strand_srn.index[downstream_3utr_intragenic_mask | downstream_5utr_intragenic_mask], 'gene_id'] = gene_row.iloc[0]



In [20]:
# Ensure numeric columns are properly formatted
srn_results.loc[:, ['start', 'end']] = srn_results[['start', 'end']].astype(int)
gene_results.loc[:, ['start', 'end']] = gene_results[['start', 'end']].astype(int)

# Classify remaining unclassified rows as 'Intergenic'
srn_results.loc[srn_results['location'].isna(), 'location'] = 'Intergenic'
srn_results.loc[srn_results['location'] == 'Intergenic', 'gene_id'] = None

# Display updated DataFrame
print(srn_results)


                   exon_id    start      end strand  length    location  \
0         srn_0010_sRNA412      312      511      +     199      5' UTR   
9           srn_0020_sRNA1     8413     8585      -     172   Antisense   
14        srn_0040_Sau6817    14262    14353      -      91  Intergenic   
17           srn_0050_Teg1    15795    16036      +     241  Intergenic   
24          srn_0060_Sau65    21713    21771      -      58   Antisense   
...                    ...      ...      ...    ...     ...         ...   
3744  srn_2020_sRNA171sRNA  2974257  2974532      +     275  Intergenic   
3745      srn_2020_sRNA171  2974289  2974352      -      63  Intergenic   
3746      srn_1930.2_rsaOF  2974383  2974438      -      55  Intergenic   
3747      srn_4610_sRNA377  2974437  2974532      -      95  Intergenic   
3749      srn_5130_Sau6728  2975498  2975576      +      78  Intergenic   

           gene_id  
0     SABB_RS00005  
9     SABB_RS00030  
14            None  
17            N

In [22]:
srn_results.to_excel('Bmb9393.xlsx', index=False)


In [23]:
srn_results

,exon_id,start,end,strand,length,location,gene_id
0,srn_0010_sRNA412,312,511,+,199,5' UTR,SABB_RS00005
9,srn_0020_sRNA1,8413,8585,-,172,Antisense,SABB_RS00030
14,srn_0040_Sau6817,14262,14353,-,91,Intergenic,None
17,srn_0050_Teg1,15795,16036,+,241,Intergenic,None
24,srn_0060_Sau65,21713,21771,-,58,Antisense,SABB_RS00080
...,...,...,...,...,...,...,...
3744,srn_2020_sRNA171sRNA,2974257,2974532,+,275,Intergenic,None
3745,srn_2020_sRNA171,2974289,2974352,-,63,Intergenic,None
3746,srn_1930.2_rsaOF,2974383,2974438,-,55,Intergenic,None
3747,srn_4610_sRNA377,2974437,2974532,-,95,Intergenic,None
